In [1]:
# surpress warning
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

/Users/larrysu/.pyenv/versions/p3lab/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
a = tf.constant(3., dtype=tf.float32)
b = tf.constant(4.)
total = a + b
print(a)
print(b.name, b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Const_1:0 Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [5]:
sess = tf.Session()
result = sess.run(total)
print(type(result), result)

<class 'numpy.float32'> 7.0


In [6]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.09624088 0.51840377 0.7166587 ]
[0.79358804 0.21751893 0.42363536]
(array([1.6901082, 1.704126 , 1.4176348], dtype=float32), array([2.6901083, 2.704126 , 2.417635 ], dtype=float32))


In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1,2], y: [3,4]}))

7.5
[4. 6.]


In [8]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [9]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [10]:
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
res = sess.run(y, {x: [[1, 2, 3], [4 ,5 ,6]]})
print(str(res))

[[0.653571 ]
 [3.1960459]]


## Feature columns

In [12]:
features = {
    'sales' : [[5],[10],[8],[9]],
    'department': ['sports','sports','gardening','gardening']}

col_department = tf.feature_column.categorical_column_with_vocabulary_list(
    'department', ['sports','gardening'])
col_department = tf.feature_column.indicator_column(col_department)
col_sales = tf.feature_column.numeric_column('sales')

columns = [col_sales, col_department]
inputs = tf.feature_column.input_layer(features, columns)
print(str(inputs))

Tensor("input_layer/concat:0", shape=(4, 3), dtype=float32)


In [13]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


## Training

$$
\hat{y} = w^T x + b
$$

`w = Dense.variables[0]`  
`b = Dense.variables[1]`

In [14]:
x = tf.constant([[1],[2],[3],[4]], dtype=tf.float32)
y = tf.constant([[0],[-1],[-2],[-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)
y_hat = linear_model(x)
print(str(linear_model))
print(str(y_hat))

Tensor("dense_1/BiasAdd:0", shape=(4, 1), dtype=float32)


In [15]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y_hat))

[[-0.2797588 ]
 [-0.5595176 ]
 [-0.83927643]
 [-1.1190352 ]]


In [16]:
loss = tf.losses.mean_squared_error(labels=y, predictions=y_hat)
print(str(loss))
print(sess.run(loss))

Tensor("mean_squared_error/value:0", shape=(), dtype=float32)
1.2893994


In [17]:
print(linear_model.weights)
print("variables len: ", len(linear_model.variables))
print("variables 1: ", linear_model.variables[0])
print("variables 2: ", linear_model.variables[1])

vs = sess.run(linear_model.variables)
print("w = ", vs[0])
print("b = ", vs[1])

[<tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float32_ref>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>]
variables len:  2
variables 1:  <tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float32_ref>
variables 2:  <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>
w =  [[-0.2797588]]
b =  [0.]


In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

print(str(optimizer))
print(str(train))

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_dense_1/kernel/ApplyGradientDescent"
input: "^GradientDescent/update_dense_1/bias/ApplyGradientDescent"



In [19]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    if i % 20 == 0 or i==99:
        print(i, " : loss = ", loss_value)

0  : loss =  1.2893994
20  : loss =  0.18518175
40  : loss =  0.16361041
60  : loss =  0.14511834
80  : loss =  0.12871669
99  : loss =  0.11485548


In [22]:
print(linear_model.weights)
print("variables len: ", len(linear_model.variables))
print("variables 1: ", linear_model.variables[0])
print("variables 2: ", linear_model.variables[1])

vs = sess.run(linear_model.variables)
print("w = ", vs[0])
print("b = ", vs[1])

y_values = sess.run(y_hat)
print("y = ", y_values)

[<tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float32_ref>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>]
variables len:  2
variables 1:  <tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float32_ref>
variables 2:  <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>
w =  [[-0.71878785]]
b =  [0.17320193]
y =  [[-0.54558593]
 [-1.2643738 ]
 [-1.9831616 ]
 [-2.7019494 ]]
